In [3]:
# !pip install openai
# !pip install python-dotenv

In [ ]:
import os
from tokenize import maybe
from dotenv import load_dotenv

load_dotenv() # 載入環境變量

my_key = os.getenv('QWEN_API_KEY') 

In [25]:
import os
from openai import OpenAI

# establish a connection to the API
client = OpenAI(
    # If the environment variable is not set, replace the following line with your Model Studio API key: api_key="sk-xxx",
    # API keys differ by region. Get your key: https://www.alibabacloud.com/help/en/model-studio/get-api-key
    api_key=os.getenv("QWEN_API_KEY"),
    # For Beijing region: https://dashscope.aliyuncs.com/compatible-mode/v1
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1",  
)

def call_qwen(system_prompt, user_prompt, client):
    '''
    This function is used to call the Qwen LLM.
    '''
    completion = client.chat.completions.create(
        # This example uses qwen-plus. You can change the model name as needed. For a list of models, see https://www.alibabacloud.com/help/en/model-studio/getting-started/models
        model="qwen-flash",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        # For Qwen3 models, use the enable_thinking parameter to control the thinking process (default is True for open source editions and False for commercial editions).
        # When using an open source edition of a Qwen3 model, if streaming output is not enabled, uncomment the following line to avoid errors.
        # extra_body={"enable_thinking": False},
    )

    return completion

Method 1 - extract component by auto batching

In [ ]:
import pandas as pd
df = pd.read_csv("C:/Users/genhk/Desktop/JDA_Tiffany/JDA/Final_Project/Bangkok/listing_name.csv", usecols=['id', 'name'])
print(len(df))

335


In [ ]:
import pandas as pd
import time
import json

batch = 100
start_row = 0
end_row = len(df)

for i in range(start_row, end_row + 1, batch):
    start_time = time.time()
    batch_df = df[['id', 'name']].iloc[i:min(i+batch, end_row + 1)]
    all_products = batch_df.to_dict(orient='records')
    print(all_products)


    system_prompt = '''
    You are a helpful marketing assistant from short-term platform. You are given a list of listing post title.  
    Your task is to analyze the structure of each post title and identify which key components are included each post title. 
    Please noted that the components "Property Type" means the type of property being listed, such as apartment, villa, condo, house, etc.
    Return the results in a structured JSON format.

    * id - e.g. 123
    * name - post title, e.g. "Cozy 2BR Apartment near BTS with Pool & Gym"
    * Title analysis - "Location Details", "Property Type", "Bedroom Count", "Amenities", "Descriptive Adjectives", "Proximity to Transit", "View", "Themed Features", "Additional Services"
    * Components counts - The count of components used in the title.

    Title analysis output format example: "Location Details", "Property Type", "Proximity to Transit"

    '''

    user_prompt = f"Here is the post listing list, with each call being presented as a member of a python list: {all_products}, "

    completion = call_qwen(system_prompt, user_prompt, client)


    try:    
        print("start")
        print(completion.model_dump_json()) # print str

        print(completion.model_dump()['choices'][0]['message']['content']) # extract content from a dict and print it  

        result = json.loads(completion.model_dump()['choices'][0]['message']['content'].replace('```json', '').replace('```', '').replace('\n', ''))
        
        #export the result to a json file
        with open(f'result_{i}.json', 'w') as f:
            json.dump(result, f)
            print(f'result_{i}.json saved in  {time.time() - start_time}s')

    except Exception as e:
        print(e)
        pass    

print("start")
print(completion.model_dump_json()) # print str

[{'id': 893157719596889523, 'name': "New！Sale! On Nut's tallest/loft/ pool/gym/BTS 300m"}, {'id': 893168506110879989, 'name': 'Family room near Donmuang airport'}, {'id': 893171666741412963, 'name': 'Big Studio Room 40 Sqm. landmark in Sathorn.'}, {'id': 893210287484507244, 'name': 'Forum Park Hotel'}, {'id': 893230441498614726, 'name': 'Cozy Room Close to Platinum Shopping Mall'}, {'id': 893235493402766569, 'name': 'Cozy Room with Bathtub Close to Platinum Mall'}, {'id': 893246775421342553, 'name': 'Cozy Room  3 PAX Close to Platinum Mall'}, {'id': 893250490793761906, 'name': 'Triple Bed at Hotel Near Platinum Fashion Mall'}, {'id': 893254035662741678, 'name': 'Friend/Family Room Connected Near Platinum Mall'}, {'id': 893347944833130332, 'name': '一室一厅公寓，小区配套设施齐全（8/612）'}, {'id': 893363969015755833, 'name': 'C7#曼谷素坤逸地铁/轻轨交汇/温馨单间/可做饭/火车夜市/ 免费泳池健身房/康民医院/ ibaby'}, {'id': 893373525609391955, 'name': 'Papaya House Mid century stay'}, {'id': 893594253146261274, 'name': 'Very large nice  1 be

Method 2 - manual batching

In [ ]:
name_2000 = list(zip(df['id'], df['name']))[200:len(df)]
    
system_prompt = '''
You are a helpful marketing assistant from short-term platform. You are given a list of listing post title.  
Your task is to analyze the structure of each post title and identify which key components are included each post title. 
Please noted that the components "Property Type" means the type of property being listed, such as apartment, villa, condo, house, etc.
Return the results in a structured JSON format.

* id - e.g. 123
* name - post title, e.g. "Cozy 2BR Apartment near BTS with Pool & Gym"
* Title analysis - "Location Details", "Property Type", "Bedroom Count", "Amenities", "Descriptive Adjectives", "Proximity to Transit", "View", "Themed Features", "Additional Services"
* Components counts - The count of components used in the title.

Title analysis output format example: "Location Details", "Property Type", "Proximity to Transit"
'''

user_prompt = f"Here is the post listing list, with each call being presented as a member of a python list: {name_2000}, "

completion = call_qwen(system_prompt, user_prompt, client)

print("start")
print(completion.model_dump_json()) # print str

start
{"id":"chatcmpl-e88b3162-d548-9034-a91f-bd9974a4cb98","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"```json\n[\n  {\n    \"id\": 46738075,\n    \"name\": \"【K10】Sky pool Luxury Condo Sathorn CBD&5mins BTS\",\n    \"Title analysis\": [\n      \"Property Type\",\n      \"Amenities\",\n      \"Proximity to Transit\",\n      \"Location Details\"\n    ],\n    \"Components counts\": 4\n  },\n  {\n    \"id\": 46738680,\n    \"name\": \"【K12】Sky pool&gym Luxury Condo in Sathon 5mins BTS\",\n    \"Title analysis\": [\n      \"Property Type\",\n      \"Amenities\",\n      \"Proximity to Transit\",\n      \"Location Details\"\n    ],\n    \"Components counts\": 4\n  },\n  {\n    \"id\": 46785564,\n    \"name\": \"NobleBE19小区，BTSAsoke+onebedroomfeewifi+gym免费网络+泳池\",\n    \"Title analysis\": [\n      \"Property Type\",\n      \"Bedroom Count\",\n      \"Amenities\",\n      \"Proximity to Transit\",\n      \"Location Details\"\n    ],\n    \"Components coun

In [69]:
import json
import re

# Step 1: 清理 LLM 回傳的字串
raw_response = completion.model_dump()['choices'][0]['message']['content']
clean_resp = raw_response.replace('```json', '').replace('```', '').replace('\n','').strip()

# Step 2: 修正非法反斜線（如果有）
safe_json_str = re.sub(r'\\(?![\"\\/bfnrtu])', r'\\\\', clean_resp)

# Step 3: 將字串轉成 Python 字典
try:
    json_output = json.loads(safe_json_str)
except json.JSONDecodeError as e:
    print("解析失敗：", e)
    # 可選：儲存原始字串以便除錯
    with open("error_raw.json", "w", encoding="utf-8") as f:
        f.write(safe_json_str)
    raise


# Step 4: 儲存成 JSON 檔案
with open(f'result_335.json', 'w', encoding='utf-8') as f:
    json.dump(json_output, f, ensure_ascii=False, indent=4)
    print(f'result_335.json saved')



result_335.json saved
